# asigna_estacion_incidentes_1.1

En este script se va a asignar a cada incidente la estacion mas cercana al lugar del mismo. El metodo va a ser cruzar ambos ficheros por fecha, estableciendo una serie de reestricciones, a saber:

    -temperatura media, maxima y minimas informadas.
    -precipitaciones informadas
    -presion informada
    -velocidad del viento informada

una vez hecho esto, lo que se hara es para cada incidente calcular la distancia a las diferentes estaciones, y agregar el registro seleccionando la distancia minima.

In [1]:
import pandas as pd
from numpy import radians
from math import sin, cos, sqrt, atan2

In [2]:
df_meteo=pd.read_csv('../../DATASETS/ESTACIONES/historico_estaciones_coordenadas_decimal.csv',sep='|',error_bad_lines=False)
df_meteo['Longitud_y']=radians(df_meteo[['Longitud_y']])
df_meteo['Latitud_y']=radians(df_meteo[['Latitud_y']])
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359612 entries, 0 to 359611
Data columns (total 16 columns):
Fecha               359612 non-null object
ID                  359612 non-null int64
Estacion            359612 non-null object
Estado_x            359612 non-null object
Latitud_y           359612 non-null float64
Longitud_y          359612 non-null float64
T. med. (ºC)        359612 non-null object
T. max (ºC)         359612 non-null object
T. min (ºC)         359612 non-null object
Prec. (mm)          359612 non-null object
Presion/ Geopot.    359612 non-null object
Dir. vi.            359612 non-null object
Vel. vi. (Km/h)     359612 non-null object
Nub.                359612 non-null object
Prof. nieve (cm)    359612 non-null object
Insolac. (horas)    283707 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 43.9+ MB


In [3]:
df_incidentes=pd.read_csv('../../DATASETS/AHOGADOS/CSV/Datos_2019-2013.csv',sep='|',error_bad_lines=False)
df_incidentes['Longitud']=radians(df_incidentes[['Longitud']])
df_incidentes['Latitud']=radians(df_incidentes[['Latitud']])
df_incidentes.head()

,Fecha,IdAhogado,IdPersona,Localidad,Provincia,CCAA,Hora,Latitud,Longitud,Sexo,...,TipoAhogamiento,Factor,Intervencion,Pronostico,Localizacion,Riesgo,Reanimacion,Vigilancia,Actividad,Deteccion
0,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,Ahogado con sintomas de aspiración de agua,"Edad, Enfermedad, Enfermedad Mental",Recuperación de cadaver,Ahogamiento mortal,Bañera,Sin datos,SVA por SEM,No vigilado,Otro,Ciudadano
1,28/12/2019,6358,6983,"VIGO, RIO LAGARES",PONTEVEDRA,GALICIA,NaN,0.736986,-0.151635,Mujer,...,Posible Ahogamiento,"Edad, Enfermedad, Enfermedad Mental",Recuperación de cadaver,Ahogamiento mortal,"Rios, canales y similares",Sin datos,Recuperacion de cadaver,No vigilado,Sin datos,Ciudadano
2,28/12/2019,6355,6978,"PAJARA, PLAYA DE JANDIA, MORRO JABLE",FUERTEVENTURA,CANARIAS,14:34,0.489523,-0.250399,Hombre,...,Ahogamiento secundario a enfermedad,"Edad, Enfermedad, Enfermedad Mental",Rescate por Socorristas o personas adiestradas...,Ahogamiento no mortal con hospitalizacion,Playas con vigilancia,Sin datos,RCP basica SOS y SVA por SEM,En horario de vigilancia,Natación recreativa,Socorrista en servicio
3,28/12/2019,6353,6976,"SAN CIIBRAO, PLAYA DO TORNO",LUGO,GALICIA,6:00,0.762673,-0.129867,Hombre,...,Ahogado con sintomas de aspiración de agua,Sin datos,Rescate por personal adiestrado con equipamiento,Ahogamiento mortal,Playas sin vigilancia,Sin datos,SVA por SEM,No vigilado,Actividad cerca del agua,Ciudadano
4,27/12/2019,6359,6984,"BARCELONA, PORT VELL",BARCELONA,CATALUÑA,2:00,0.722141,0.038073,Hombre,...,Ahogamiento secundario a accidente,Sin datos,Recuperación de cadaver,Ahogamiento mortal,"Puertos, embarcaderos, amarres",Buenas condiciones metereológicas o del agua (...,Recuperacion de cadaver,No vigilado,Actividad cerca del agua,"Acompañante, familiar, amigo"


In [4]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359612 entries, 0 to 359611
Data columns (total 16 columns):
Fecha               359612 non-null object
ID                  359612 non-null int64
Estacion            359612 non-null object
Estado_x            359612 non-null object
Latitud_y           359612 non-null float64
Longitud_y          359612 non-null float64
T. med. (ºC)        359612 non-null object
T. max (ºC)         359612 non-null object
T. min (ºC)         359612 non-null object
Prec. (mm)          359612 non-null object
Presion/ Geopot.    359612 non-null object
Dir. vi.            359612 non-null object
Vel. vi. (Km/h)     359612 non-null object
Nub.                359612 non-null object
Prof. nieve (cm)    359612 non-null object
Insolac. (horas)    283707 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 43.9+ MB


In [5]:
df_meteo=df_meteo.rename(columns={"T. med. (ºC)": "T_med","T. max (ºC)": "T_max","T. min (ºC)": "T_min","Prec. (mm)": "Precipitaciones","Presion/ Geopot.": "Presion","Vel. vi. (Km/h)": "V_Viento"})


In [6]:
df_meteo=df_meteo[df_meteo.T_med != '-']
#df_meteo=df_meteo[df_meteo.T_max != '-']
#df_meteo=df_meteo[df_meteo.T_min != '-']
df_meteo=df_meteo[df_meteo.Precipitaciones != '-']
#df_meteo=df_meteo[df_meteo.Presion != '-']
df_meteo=df_meteo[df_meteo.V_Viento != '-']

df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 337023 entries, 0 to 359597
Data columns (total 16 columns):
Fecha               337023 non-null object
ID                  337023 non-null int64
Estacion            337023 non-null object
Estado_x            337023 non-null object
Latitud_y           337023 non-null float64
Longitud_y          337023 non-null float64
T_med               337023 non-null object
T_max               337023 non-null object
T_min               337023 non-null object
Precipitaciones     337023 non-null object
Presion             337023 non-null object
Dir. vi.            337023 non-null object
V_Viento            337023 non-null object
Nub.                337023 non-null object
Prof. nieve (cm)    337023 non-null object
Insolac. (horas)    265495 non-null object
dtypes: float64(2), int64(1), object(13)
memory usage: 43.7+ MB


In [7]:
df_inc_meteo = pd.merge(df_incidentes, 
                  df_meteo,
                  left_on=['Fecha'],
                  right_on=['Fecha'],
                  how='left')

In [8]:
df_inc_meteo.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865914 entries, 0 to 865913
Data columns (total 41 columns):
Fecha               865914 non-null object
IdAhogado           865914 non-null int64
IdPersona           865914 non-null int64
Localidad           865914 non-null object
Provincia           865914 non-null object
CCAA                865914 non-null object
Hora                693181 non-null object
Latitud             865914 non-null float64
Longitud            865914 non-null float64
Sexo                865845 non-null object
Edad                703569 non-null float64
Nacionalidad        865914 non-null object
Origen              865914 non-null object
Extraccion          865914 non-null object
Titular             865914 non-null object
Causa               865914 non-null object
TipoAhogamiento     865914 non-null object
Factor              865914 non-null object
Intervencion        865914 non-null object
Pronostico          865914 non-null object
Localizacion        865914 n

In [9]:
def calcular_distancia(lon1,lat1,lon2,lat2):

    R = 6373.0

    dlon = lon2 - lon1

    dlat = lat2 - lat1

    a = (sin(dlat/2))**2 + cos(lat1) * cos(lat2) * (sin(dlon/2))**2

    c = 2 * atan2(sqrt(a), sqrt(1-a))

    distance = R * c

    #print ("Result", str(distance))

    return distance

In [10]:
df_inc_meteo['incremental'] = df_inc_meteo.index

In [11]:
df_inc_meteo.head()

,Fecha,IdAhogado,IdPersona,Localidad,Provincia,CCAA,Hora,Latitud,Longitud,Sexo,...,T_max,T_min,Precipitaciones,Presion,Dir. vi.,V_Viento,Nub.,Prof. nieve (cm),Insolac. (horas),incremental
0,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,16.4,8.4,0.0,1030.8 Hpa,164º(S),7,2/8,-,8.4,0
1,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,15.9,3.7,0.0,1030.8 Hpa,165º(S),9,-,-,8.2,1
2,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,14.8,10.9,0.0,1028.2 Hpa,258º(W),11,-,-,-,2
3,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,14.7,10.4,0.0,-,188º(S),15,-,-,-,3
4,30/12/2019,6360,6985,GIRONA,GIRONA,CATALUÑA,14:30,0.732767,0.049177,Mujer,...,13.4,-2.4,0.0,1033.3 Hpa,143º(SE),3,0/8,-,5.5,4


In [12]:
df_inc_meteo_calculaDistancia = dict()

for num_linea, j in df_inc_meteo.iterrows():
    
    incremental=j[41]
    
    id_persona=j[2]
    
    fecha=j[0]

    i_longitud=j[8]

    i_latitud=j[7]
    
    id_estacion=j[26]
    
    e_longitud=j[30]

    e_latitud=j[29]

    #rint(num_linea)
    
    e_distancia=calcular_distancia(i_longitud,i_latitud,e_longitud,e_latitud)

    #rint("ID AHOGADO: " + str(id_persona) +" , Fecha: " + str(fecha)+ " , LONGITUD: "  + str(i_longitud) + " , LATITUD: " + str(i_latitud) + " , id_estacion: " + str(id_estacion) + " , LONGITUD: "  + str(e_longitud) + " , LATITUD: " + str(e_latitud))

    distancia = e_distancia
    
    #rint("Distancia: "+ str(distancia))
    
    estacion_number = id_estacion

    df_inc_meteo_calculaDistancia[incremental]=fecha,id_persona,estacion_number,distancia

 

df_inc_meteo_calculaDistancia

{0: ('30/12/2019', 6985, 8001, 930.7333175237114),
 1: ('30/12/2019', 6985, 8002, 927.3444537875805),
 2: ('30/12/2019', 6985, 8004, 878.3714890036248),
 3: ('30/12/2019', 6985, 8006, 993.0186322533976),
 4: ('30/12/2019', 6985, 8008, 850.1556114756938),
 5: ('30/12/2019', 6985, 8009, 821.0339941128198),
 6: ('30/12/2019', 6985, 8011, 743.0163423868682),
 7: ('30/12/2019', 6985, 8014, 716.9036886652041),
 8: ('30/12/2019', 6985, 8015, 725.963988527075),
 9: ('30/12/2019', 6985, 8016, 638.4635962663265),
 10: ('30/12/2019', 6985, 8017, 609.6726537012603),
 11: ('30/12/2019', 6985, 8020, 518.8826352667846),
 12: ('30/12/2019', 6985, 8021, 565.4999992708711),
 13: ('30/12/2019', 6985, 8023, 566.426101826601),
 14: ('30/12/2019', 6985, 8025, 492.6536035207035),
 15: ('30/12/2019', 6985, 8026, 439.7103844298022),
 16: ('30/12/2019', 6985, 8027, 424.2229812856095),
 17: ('30/12/2019', 6985, 8029, 406.99264741252),
 18: ('30/12/2019', 6985, 8030, 379.43768593014784),
 19: ('30/12/2019', 6985,

In [13]:
df_distancias=pd.DataFrame.from_dict(df_inc_meteo_calculaDistancia, orient='index')

df_distancias.columns = ['fecha','IdPersona','ID_Estacion','Distancia']

In [14]:
df_distancias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865914 entries, 0 to 865913
Data columns (total 4 columns):
fecha          865914 non-null object
IdPersona      865914 non-null int64
ID_Estacion    865914 non-null int64
Distancia      865914 non-null float64
dtypes: float64(1), int64(2), object(1)
memory usage: 33.0+ MB


In [15]:
df_inc_meteo_distancia = pd.merge(df_inc_meteo, 
                  df_distancias,
                  left_on=['Fecha','IdPersona','ID'],
                  right_on=['fecha','IdPersona','ID_Estacion'],
                  how='left')

In [16]:
df_inc_meteo_distancia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 865914 entries, 0 to 865913
Data columns (total 45 columns):
Fecha               865914 non-null object
IdAhogado           865914 non-null int64
IdPersona           865914 non-null int64
Localidad           865914 non-null object
Provincia           865914 non-null object
CCAA                865914 non-null object
Hora                693181 non-null object
Latitud             865914 non-null float64
Longitud            865914 non-null float64
Sexo                865845 non-null object
Edad                703569 non-null float64
Nacionalidad        865914 non-null object
Origen              865914 non-null object
Extraccion          865914 non-null object
Titular             865914 non-null object
Causa               865914 non-null object
TipoAhogamiento     865914 non-null object
Factor              865914 non-null object
Intervencion        865914 non-null object
Pronostico          865914 non-null object
Localizacion        865914 n

In [17]:
df_inc_meteo_distancia=df_inc_meteo_distancia.sort_values(by=['IdPersona', 'Distancia','Fecha'])

In [18]:
df_inc_meteo_distancia.head()

,Fecha,IdAhogado,IdPersona,Localidad,Provincia,CCAA,Hora,Latitud,Longitud,Sexo,...,Presion,Dir. vi.,V_Viento,Nub.,Prof. nieve (cm),Insolac. (horas),incremental,fecha,ID_Estacion,Distancia
865907,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1022.7 Hpa,340º(N),3,6/8,-,1.2,865907,01/01/2013,8419,47.562655
865912,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1021.2 Hpa,288º(W),17,6/8,-,5.3,865912,01/01/2013,8482,67.801181
865906,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1023.1 Hpa,58º(NE),2,7/8,-,0.0,865906,01/01/2013,8410,80.952206
865905,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1022.0 Hpa,74º(E),8,6/8,-,3.2,865905,01/01/2013,8397,116.278371
865904,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.07525,Hombre,...,1022.8 Hpa,44º(NE),5,7/8,-,4.7,865904,01/01/2013,8391,141.546413


In [19]:
df_inc_meteo_distancia=df_inc_meteo_distancia.drop_duplicates(subset=['Fecha','IdAhogado','IdPersona','Localidad','Provincia','CCAA','Hora','Latitud','Longitud','Sexo','Edad','Nacionalidad','Origen','Extraccion','Titular','Causa','TipoAhogamiento','Factor','Intervencion','Pronostico','Localizacion','Riesgo','Reanimacion','Vigilancia','Actividad','Deteccion'],keep='first')

In [20]:
df_inc_meteo_distancia.head()

,Fecha,IdAhogado,IdPersona,Localidad,Provincia,CCAA,Hora,Latitud,Longitud,Sexo,...,Presion,Dir. vi.,V_Viento,Nub.,Prof. nieve (cm),Insolac. (horas),incremental,fecha,ID_Estacion,Distancia
865907,01/01/2013,144,166,IZNAJAR,CORDOBA,ANDALUCIA,9:40,0.650319,-0.075250,Hombre,...,1022.7 Hpa,340º(N),3,6/8,-,1.2,865907,01/01/2013,8419,47.562655
865686,09/01/2013,146,168,PUERTO DE LA CRUZ,TENERIFE,CANARIAS,15:36,0.495824,-0.288784,Hombre,...,1028.7 Hpa,347º(N),5,1/8,-,7.9,865686,09/01/2013,8451,1349.441373
865854,05/01/2013,147,169,ARRECIFE,LANZAROTE,CANARIAS,16:00,0.504845,-0.237980,Hombre,...,1033.9 Hpa,350º(N),3,0/8,-,-,865854,05/01/2013,8451,1120.630572
865798,05/01/2013,148,170,SAN BARTOLOME DE TIRAJANA,GRAN CANARIA,CANARIAS,15:35,0.484508,-0.271814,Hombre,...,1033.9 Hpa,350º(N),3,0/8,-,-,865798,05/01/2013,8451,1339.953401
865742,06/01/2013,150,172,PLAYA DEL INGLES,GRAN CANARIA,CANARIAS,15:35,0.484488,-0.271640,Hombre,...,1030.9 Hpa,315º(NW),2,1/8,-,-,865742,06/01/2013,8451,1339.428023


In [21]:
df_inc_meteo_distancia=df_inc_meteo_distancia.drop(['fecha', 'incremental','Titular','ID_Estacion'], axis=1)
df_inc_meteo_distancia.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6482 entries, 865907 to 82
Data columns (total 41 columns):
Fecha               6482 non-null object
IdAhogado           6482 non-null int64
IdPersona           6482 non-null int64
Localidad           6482 non-null object
Provincia           6482 non-null object
CCAA                6482 non-null object
Hora                5381 non-null object
Latitud             6482 non-null float64
Longitud            6482 non-null float64
Sexo                6481 non-null object
Edad                5366 non-null float64
Nacionalidad        6482 non-null object
Origen              6482 non-null object
Extraccion          6482 non-null object
Causa               6482 non-null object
TipoAhogamiento     6482 non-null object
Factor              6482 non-null object
Intervencion        6482 non-null object
Pronostico          6482 non-null object
Localizacion        6482 non-null object
Riesgo              6482 non-null object
Reanimacion         6482 non

In [22]:
df_inc_meteo_distancia.to_csv('../../DATASETS/AHOGADOS/CSV/historico_incidentes_estaciones_meteoSinPresionTminmax.csv',sep='|',header=True,index=False)